# Rat Sightings Data Cleaning

This notebook cleans the data in scr/data/rat_sightings_data/Rat_Sightings_NYC.csv and saves the results to sc/data/Rat_Sightings_Cleaned.csv. The main steps of our cleaning process is summaried here.

1. We first cleaned up the column names by making words lowercase and replacing spaces by underscores.

2. We updated all entries in the data which are dates to pandas datetime format.

3. We dropped columns with only 1 unique value since they provide no information for later analysis.

4. We cleaned up the values of the location_type columns by shortening the strings. We used a dictionary and a mapping to keep track of the changes.

5. We consolidated redundant columns such as park_borough being the same as borough.

6. We checked that the location column was redundant due to the prescence of the latitude and longitude column. Then, we dropped the location column.

7. The rat sightings data includes information on when the case was opened and closed. We added a column which tracks the number of days it took to close each case. Doing this, we found two entries with a close date occuring before the open date.

8. The incident_zip column contains the value 12345 which does not correspond to a zip code in NYC. We update the incident_zip by using the latitude and longitude columns. 

9. We save the cleaned data to a new csv file for later use. At the end, we quantified the missingness of the data using missingno's matrix and heatmap.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [ ]:
rat_sighting = pd.read_csv("../data/rat_sightings_data/Rat_Sightings_NYC.csv")
rat_sighting.columns

In [ ]:
# make letters lowercase, replace spaces with underscores, get rid of text after '(' etc

rat_sighting.columns = [t.partition('(')[0].strip().lower().replace(' ', '_') for t in rat_sighting.columns] #apply to column headers
rat_sighting['location_type'] = rat_sighting['location_type'].str.strip().str.replace(' ', '_').str.lower()  #apply to location_type column

In [ ]:
# identify columns with only 1 unique value (including NaNs)
cols_to_drop = [c for c in rat_sighting.columns if (rat_sighting[c].nunique(dropna=False) == 1)]

# Drop them all in one go
rat_sighting = rat_sighting.drop(columns=cols_to_drop)

In [ ]:
# make the datetime the correct format
rat_sighting['created_date'] = pd.to_datetime(rat_sighting['created_date']) 
rat_sighting['closed_date'] = pd.to_datetime(rat_sighting['closed_date'])
rat_sighting['resolution_action_updated_date'] = pd.to_datetime(rat_sighting['resolution_action_updated_date'])


In [ ]:
rat_sighting['location_type'].value_counts()

In [ ]:
# Create a dictionary of the "wrong" names and the "right" name
mapping = {
    '3+_family_apartment_building': '3+_family_apt._building',
    '3+family_apt.': '3+_family_apt._building',
    '3+_family_apt.': '3+_family_apt._building',
    '3+_family_apt': '3+_family_apt._building',
    'residential_building': '3+_family_apt._building',
    'residence': '3+_family_apt._building',
    'apartment': '3+_family_apt._building',
    '1-2_familydwelling': '1-2_family_dwelling',
    'school': 'school/pre-school/nursery',
    'school/pre-school': 'school/pre-school/nursery',
    'day_care_or_nursery': 'school/pre-school/nursery',
    'day_care/nursery': 'school/pre-school/nursery',
    'street':'street_area',
    'restaurant': 'restaurant/bar/deli/bakery',
    'catch_basin_or_sewer': 'catch_basin/sewer',
    'parking_lot_or_garage': 'parking_lot/garage',
    'government_building': 'office/government_building',
    'office/government_ building': 'office/government_building',
    'other_(explain_below)': 'other'
}

# Apply the fix
rat_sighting['location_type'] = rat_sighting['location_type'].replace(mapping)

In [ ]:
rat_sighting['location_type'].value_counts()

In [ ]:
# check if park_borough and borough columns are redundant
print(rat_sighting['park_borough'].equals(rat_sighting['borough']))

In [ ]:
rat_sighting = rat_sighting.drop(columns='park_borough')

In [ ]:
#notice that information from lat, lon are repeated in point

# 1. Update the mask to ensure both lat and lon are present
not_null_mask = (
    rat_sighting['location'].notnull() & 
    rat_sighting['longitude'].notnull() & 
    rat_sighting['latitude'].notnull()
)

# 2. Extract BOTH lon and lat from the POINT string
# POINT (-73.9685 40.7540) -> index 0 is lon, index 1 is lat
coords_from_point = rat_sighting['location'].str.extract(r'POINT \(([^ ]+) ([^)]+)\)').astype(float)

# 3. Check if extracted lon matches 'longitude' AND extracted lat matches 'latitude'
lon_matches = np.isclose(coords_from_point.loc[not_null_mask, 0], rat_sighting.loc[not_null_mask, 'longitude'], atol=1e-4)
lat_matches = np.isclose(coords_from_point.loc[not_null_mask, 1], rat_sighting.loc[not_null_mask, 'latitude'], atol=1e-4)

# 4. Combine them: True only if BOTH match
final_matches = lon_matches & lat_matches

print(pd.Series(final_matches).value_counts())

In [ ]:
# Drop the `location' column since it is redundant with latitude and longitude.

rat_sighting = rat_sighting.drop(columns=['location'])

In [ ]:
# 1. Strip out the word 'Unspecified' and extra spaces from the community board column
cleaned_board = rat_sighting['community_board'].str.replace('Unspecified', '', case=False).str.strip()

# 2. Extract the borough name from what remains (e.g., '03 BRONX' -> 'BRONX')
extracted_borough_cleaned = cleaned_board.str.extract(r'\d*\s*(.*)')

# 3. Re-run the comparison
matches_new = (extracted_borough_cleaned[0] == rat_sighting['borough'])

# 4. Check the results
print(pd.Series(matches_new).value_counts())

In [ ]:
# Change 'matches' to 'matches_new' to see the actual errors
mismatches = rat_sighting[matches_new == False]

# Show the columns side-by-side
mismatches[['community_board', 'borough']].head(20)

In [ ]:
# Create a mask for rows where both dates exist
both_dates_exist = rat_sighting['created_date'].notnull() & rat_sighting['closed_date'].notnull()

# Compare only the valid rows
real_date_mismatches = rat_sighting[
    (rat_sighting['created_date'] != rat_sighting['closed_date']) & 
    both_dates_exist
]

print(f"Actual mismatches (excluding NaT): {len(real_date_mismatches)}")
real_date_mismatches[['created_date', 'closed_date']].head()

In [ ]:
closed_count = rat_sighting['closed_date'].count()
closed_count

In [ ]:
print(rat_sighting['status'].value_counts())

In [ ]:
# Create a filter for rows where status is exactly 'Unspecified'
unspecified_status_rows = rat_sighting[rat_sighting['status'] == 'Unspecified']

# Print the resulting rows
unspecified_status_rows

In [ ]:
# Filter for status 'In Progress' that also have a closed_date
in_progress_with_dates = rat_sighting[
    (rat_sighting['status'] == 'In Progress') & 
    (rat_sighting['closed_date'].notnull())
]

# Display the findings
in_progress_with_dates[['unique_key', 'status', 'created_date', 'closed_date']]

In [ ]:
# Find rows where status is 'In Progress' but a closed_date exists
status_mismatch = rat_sighting[
    (rat_sighting['status'] == 'In Progress') & 
    (rat_sighting['closed_date'].notnull())
]

print(f"Rows that are 'In Progress' but have a date: {len(status_mismatch)}")
status_mismatch[['unique_key', 'status', 'created_date', 'closed_date']]

In [ ]:
# 1. Create the column first
rat_sighting['days_to_close'] = (rat_sighting['closed_date'] - rat_sighting['created_date']).dt.days

# 2. Now you can filter for negative values
time_travelers = rat_sighting[rat_sighting['days_to_close'] < 0]

# 3. Display the results
time_travelers[['unique_key', 'created_date', 'closed_date', 'days_to_close']]

# Filter for rows where the math resulted in a negative number
time_travelers = rat_sighting[rat_sighting['days_to_close'] < 0]

# Display the key columns to see the date conflict
time_travelers[['unique_key', 'created_date', 'closed_date', 'days_to_close']]

In [ ]:
# 1. Filter for only the 'Closed' status rows
closed_only = rat_sighting[rat_sighting['status'] == 'Closed'].copy()

# 2. Recalculate the days to close for this subset
closed_only['days_to_close'] = (
    (closed_only['closed_date'] - closed_only['created_date']).dt.total_seconds() / 86400
)

# 3. See the summary
print(closed_only['days_to_close'].describe())

In [ ]:
# The zip code 12345 are invalid. 
rat_sighting[rat_sighting['incident_zip']== 12345]

In [ ]:
# we fix the zip_code issue for these values

from scipy.spatial import cKDTree

# load the zip_code data
zip_db = pd.read_csv("map_data_for_cleaning/uszips.csv")
zip_db = zip_db[['zip', 'lat', 'lng']].dropna()

# Remove invalid (NaN or inf) coordinates
zip_db = zip_db[np.isfinite(zip_db['lat']) & np.isfinite(zip_db['lng'])]

# Build KDTree
tree = cKDTree(zip_db[['lat', 'lng']].values)

def nearest_zip(lat, lon):
    """Return ZIP code nearest to a given latitude/longitude."""
    if not np.isfinite(lat) or not np.isfinite(lon):
        return pd.NA  # skip invalid coordinates
    distance, idx = tree.query([lat, lon])
    return int(zip_db.iloc[idx]['zip'])

# fix problematic rows
zip_codes_to_fix = {12345}
mask = rat_sighting['incident_zip'].isin(zip_codes_to_fix)

# Only apply to rows with valid lat/lon
valid_mask = mask & rat_sighting['latitude'].notna() & rat_sighting['longitude'].notna()

rat_sighting.loc[valid_mask, 'incident_zip'] = rat_sighting.loc[valid_mask].apply(
    lambda r: nearest_zip(r['latitude'], r['longitude']),
    axis=1
)

In [ ]:
# we will use ZIP codes quite frequently, so it is convenient to change incident_zip to zip.

rat_sighting.rename(columns={'incident_zip':'zip'}, inplace=True)

In [ ]:
# Save cleaned up rat_sighting data to a new CSV file

rat_sighting.to_csv("../data/cleaned_rat_sightings_data/cleaned_rat_sightings.csv", index=False)


In [ ]:
# Here's a sample of the cleaned up data

rat_sighting.sample(5)

In [ ]:
# We check missingness of the data.

msno.matrix(rat_sighting)
msno.heatmap(rat_sighting)